In [68]:
import psycopg2
import pandas as pd 
# from sqlalchemy.types import Integer, Text, String, DateTime
import sqlalchemy as s
import matplotlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json

with open("config.json") as config_file:
    config = json.load(config_file)

database_connection_string = 'postgres+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = s.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

In [69]:
missing_query = f"""
SELECT
    * 
FROM
    (
    SELECT
        repo_info.repo_id,
        repo.repo_name,
repo.repo_git, 
        MAX ( pull_request_count ) AS max_pr_count,
        COUNT ( * ) AS meta_count 
    FROM
        repo_info,
        repo -- WHERE issues_enabled = 'true' 
    WHERE
        pull_request_count >= 1
        AND repo.repo_id = repo_info.repo_id 
    GROUP BY
        repo_info.repo_id,
        repo.repo_name, 
 repo.repo_git 
    ORDER BY
        repo_info.repo_id,
        repo.repo_name 
    ) yy
    LEFT OUTER JOIN (
    SELECT A
        .repo_id,
        A.repo_name,
        b.pull_request_count,
        d.repo_id AS pull_request_repo_id,
        e.last_collected,
        f.last_pr_collected,
        COUNT ( * ) AS pull_requests_collected,
        ( b.pull_request_count - COUNT ( * ) ) AS pull_requests_missing,
        ABS ( CAST ( ( COUNT ( * ) ) + 1 AS DOUBLE PRECISION ) / CAST ( b.pull_request_count + 1 AS DOUBLE PRECISION ) ) AS ratio_abs,
        ( CAST ( ( COUNT ( * ) ) + 1 AS DOUBLE PRECISION ) / CAST ( b.pull_request_count + 1 AS DOUBLE PRECISION ) ) AS ratio_prs 
    FROM
        augur_data.repo A,
        augur_data.pull_requests d,
        augur_data.repo_info b,
        ( SELECT repo_id, MAX ( data_collection_date ) AS last_collected FROM augur_data.repo_info GROUP BY repo_id ORDER BY repo_id ) e,
        ( SELECT repo_id, MAX ( data_collection_date ) AS last_pr_collected FROM augur_data.pull_requests GROUP BY repo_id ORDER BY repo_id ) f 
    WHERE
        A.repo_id = b.repo_id 
        AND LOWER ( A.repo_git ) LIKE'%github.com%' 
        AND A.repo_id = d.repo_id 
        AND b.repo_id = d.repo_id 
        AND e.repo_id = A.repo_id 
        AND b.data_collection_date = e.last_collected 
        AND f.repo_id = A.repo_id -- AND d.pull_request_id IS NULL
        
    GROUP BY
        A.repo_id,
        d.repo_id,
        b.pull_request_count,
        e.last_collected,
        f.last_pr_collected 
    ORDER BY
        ratio_abs desc
    ) zz ON yy.repo_id = zz.repo_id 
ORDER BY
    ratio_abs;
    """
missing_data = pd.read_sql_query(missing_query, con=engine)
print(missing_data)

      repo_id                    repo_name  \
0       27548                      pairist   
1       28108                  oxygen-mask   
2       28141                boarding-pass   
3       30836            projectreactor.io   
4       30223        bitnami-docker-pgpool   
...       ...                          ...   
2237    26440                     docs-ksm   
2238    26291      datadog-firehose-nozzle   
2239    25565  bosh-linux-stemcell-builder   
2240    25579                      netdata   
2241    27175           cf-SpringCloud-IoT   

                                               repo_git  max_pr_count  \
0                 https://github.com/pivotal-cf/pairist            14   
1              https://github.com/concourse/oxygen-mask            14   
2            https://github.com/concourse/boarding-pass            12   
3          https://github.com/reactor/projectreactor.io            39   
4      https://github.com/bitnami/bitnami-docker-pgpool            19   
...      

In [70]:
df = missing_data.loc[missing_data['ratio_abs'] < 1]
#print(df)
df1 = df[['repo_id', 'last_pr_collected', 'pull_requests_missing', 'ratio_abs', 'repo_git']]
#df1 = df[['pull_requests_missing', 'repo_git']]
print(df1)
#missing_data.to_csv('/Users/dafoster/gitrepos/augur-pivotal/code/health_scripts/output/missing.csv')

    repo_id  repo_id   last_pr_collected  pull_requests_missing  ratio_abs  \
0     27548  27548.0 2021-01-22 04:09:19                    2.0   0.866667   
1     28108  28108.0 2020-12-07 18:49:06                    2.0   0.866667   
2     28141  28141.0 2020-12-07 18:57:41                    1.0   0.923077   
3     30836  30836.0 2020-12-10 15:25:29                    3.0   0.925000   
4     30223  30223.0 2020-12-07 20:00:01                    1.0   0.950000   
..      ...      ...                 ...                    ...        ...   
69    28037  28037.0 2021-01-22 06:13:33                    1.0   0.999096   
70    27513  27513.0 2021-01-22 09:38:36                    1.0   0.999123   
71    30898  30898.0 2021-01-22 09:39:03                    1.0   0.999153   
72    30000  30000.0 2021-01-22 11:52:50                    1.0   0.999495   
73    25857  25857.0 2021-01-22 10:05:05                    3.0   0.999689   

                                            repo_git  
0       

In [71]:
df = missing_data.loc[missing_data['ratio_abs'] < 1]
#print(df)
df1 = df[['repo_id', 'last_pr_collected', 'pull_requests_missing', 'ratio_abs', 'repo_git']]
#df1 = df[['pull_requests_missing', 'repo_git']]
print(df1)

for index, repo in df1.iterrows():
    if 'saltstack' in repo['repo_git']:
        print(repo['ratio_abs'], repo['repo_git'])

    repo_id  repo_id   last_pr_collected  pull_requests_missing  ratio_abs  \
0     27548  27548.0 2021-01-22 04:09:19                    2.0   0.866667   
1     28108  28108.0 2020-12-07 18:49:06                    2.0   0.866667   
2     28141  28141.0 2020-12-07 18:57:41                    1.0   0.923077   
3     30836  30836.0 2020-12-10 15:25:29                    3.0   0.925000   
4     30223  30223.0 2020-12-07 20:00:01                    1.0   0.950000   
..      ...      ...                 ...                    ...        ...   
69    28037  28037.0 2021-01-22 06:13:33                    1.0   0.999096   
70    27513  27513.0 2021-01-22 09:38:36                    1.0   0.999123   
71    30898  30898.0 2021-01-22 09:39:03                    1.0   0.999153   
72    30000  30000.0 2021-01-22 11:52:50                    1.0   0.999495   
73    25857  25857.0 2021-01-22 10:05:05                    3.0   0.999689   

                                            repo_git  
0       

In [72]:
for index, repo in df1.iterrows():
    print(repo['last_pr_collected'], repo['repo_git'])

2021-01-22 04:09:19 https://github.com/pivotal-cf/pairist
2020-12-07 18:49:06 https://github.com/concourse/oxygen-mask
2020-12-07 18:57:41 https://github.com/concourse/boarding-pass
2020-12-10 15:25:29 https://github.com/reactor/projectreactor.io
2020-12-07 20:00:01 https://github.com/bitnami/bitnami-docker-pgpool
2021-01-19 16:50:58 https://github.com/saltstack/community
2020-12-07 18:32:54 https://github.com/vmware-tanzu/sonobuoy-plugins
2020-12-27 15:23:04 https://github.com/projectriff/node-function-invoker
2021-01-15 15:34:38 https://github.com/vmware/nsx-powerops
2021-01-22 08:45:12 https://github.com/k14s/ytt
2020-12-27 15:46:49 https://github.com/greenplum-db/gpbackup
2020-12-27 13:56:32 https://github.com/vmware/flowgate
2020-12-27 15:02:17 https://github.com/vmware/container-service-extension
2020-12-27 14:45:02 https://github.com/vmware/differential-datalog
2021-01-15 19:53:42 https://github.com/vmware-tanzu/velero-plugin-for-microsoft-azure
2020-12-27 15:45:36 https://githu

In [73]:
#Repos for gathering data

import datetime 

current = datetime.date.today()
today = "'" + str(current) + "'"
#print(current)

first_current = current.replace(day=1)
last_month = first_current - datetime.timedelta(days=1)
end_date = "'" + str(last_month) + "'"

start = last_month - datetime.timedelta(days=180)
start_date = "'" + str(start) + "'"


end_date = today
print(start_date, end_date)

def get_commits_by_repo(start_date, end_date, engine):
    import pandas as pd

    repo_list_commits = pd.DataFrame()
    repo_list_commits_query = f"""
            SELECT COUNT(DISTINCT commits.cmt_commit_hash), repo.repo_id, repo.repo_name, repo.repo_path from repo, commits
            WHERE 
                repo.repo_id = commits.repo_id
                AND commits.cmt_author_timestamp >= {start_date}
                AND commits.cmt_author_timestamp <= {end_date}
                AND cmt_author_name NOT LIKE '%%utomation%%'
                AND cmt_author_name NOT LIKE '%%ipeline%%'
                AND cmt_author_name NOT LIKE '%%Cloud Foundry%%'
                AND cmt_author_name NOT LIKE 'snyk%%'
                AND cmt_author_name NOT LIKE '%%bot'
                AND cmt_author_name NOT LIKE 'dependabot%%'
                AND cmt_author_name NOT LIKE '%%Bot'
                AND cmt_author_name NOT LIKE '%%BOT'
                AND cmt_author_name != 'cfcr'
                AND cmt_author_name != 'CFCR'
                AND cmt_author_name != 'Travis CI'
                AND cmt_author_name != 'Cloud Foundry London'
                AND cmt_author_name != 'pivotal-rabbitmq-ci'
                AND cmt_author_name != 'Bitnami Containers'
                AND cmt_author_name != 'Spring Operator'
                AND cmt_author_name != 'Spring Buildmaster'
            GROUP BY repo.repo_id
            ORDER BY COUNT(DISTINCT commits.cmt_commit_hash);
            """
    repo_list_commits = pd.read_sql_query(repo_list_commits_query, con=engine)
    
    return repo_list_commits
    
repo_list_commits = get_commits_by_repo(start_date, end_date, engine)

top = repo_list_commits.loc[repo_list_commits['count'] > 60]



'2020-07-04' '2021-01-25'


In [74]:
print(top)

      count  repo_id             repo_name  \
1157     61    27761  docs-utility-scripts   
1158     62    27526    docs-operating-pas   
1159     62    27939          nsx-powerops   
1160     63    27435      docs-cloud-cache   
1161     63    30820      reactor-rabbitmq   
...     ...      ...                   ...   
1307   1180    28051             concourse   
1308   1492    30932                  salt   
1309   1590    25432       rabbitmq-server   
1310   3114    25760           spring-boot   
1311   4545    27645               cvelist   

                                repo_path  
1157  github.com/pivotal-cf-experimental/  
1158               github.com/pivotal-cf/  
1159                   github.com/vmware/  
1160               github.com/pivotal-cf/  
1161                  github.com/reactor/  
...                                   ...  
1307                github.com/concourse/  
1308                github.com/saltstack/  
1309                 github.com/rabbitmq/  
1310   

In [75]:
print('repo_id ratio last_pr_collected              repo_url')
i = 0
for index, repo in top.iterrows():
    repo_id = repo['repo_id']

    try:
        #ratio_abs = round(missing_data.loc[missing_data['pull_request_repo_id'] == repo_id]['ratio_abs'].values[0], 2)
        ratio_abs = format(missing_data.loc[missing_data['pull_request_repo_id'] == repo_id]['ratio_abs'].values[0], '.2f')
    except:
        ratio_abs = 'NA  '
    
    try:
        last_pr = missing_data.loc[missing_data['pull_request_repo_id'] == repo_id]['last_pr_collected'].values[0]
    except:
        last_pr = 'UNKNOWN'
        
    repo_info = str(repo_id) + '   ' + str(ratio_abs) + '  ' + str(last_pr) + '  ' + repo['repo_path'] + repo['repo_name'] 
    print(repo_info)
    i+=1
    
print('Number of repos:', i)

repo_id ratio last_pr_collected              repo_url
27761   1.00  2020-12-07T17:18:24.000000000  github.com/pivotal-cf-experimental/docs-utility-scripts
27526   1.00  2021-01-15T18:22:55.000000000  github.com/pivotal-cf/docs-operating-pas
27939   0.97  2021-01-15T15:34:38.000000000  github.com/vmware/nsx-powerops
27435   1.00  2020-12-27T16:05:21.000000000  github.com/pivotal-cf/docs-cloud-cache
30820   1.00  2020-12-27T14:56:07.000000000  github.com/reactor/reactor-rabbitmq
25807   1.00  2021-01-15T20:08:35.000000000  github.com/Spring-projects/spring-data-keyvalue
30281   1.00  2021-01-22T07:37:35.000000000  github.com/carbonblack/cbapi-python
30434   1.00  2021-01-15T17:22:08.000000000  github.com/vmware-samples/rabbitmq-oauth-example-app
28028   1.00  2020-12-27T11:15:07.000000000  github.com/vmware/captive-web-view
30660   1.00  2020-12-27T16:54:23.000000000  github.com/spring-cloud/spring-cloud-build
30334   1.00  2021-01-15T15:39:51.000000000  github.com/vmware-labs/service-bi

30478   1.00  2021-01-15T10:57:58.000000000  github.com/k14s/k14s.io
25880   1.00  2021-01-22T10:25:39.000000000  github.com/greenplum-db/gpupgrade
30386   1.00  2021-01-22T05:51:05.000000000  github.com/saltstack/pytest-salt-factories
25827   1.00  2020-12-07T08:52:35.000000000  github.com/Spring-projects/sts4
27512   1.00  2021-01-22T04:27:21.000000000  github.com/pivotal-cf/docs-ops-manager
30455   1.00  2021-01-22T07:33:27.000000000  github.com/kubeapps/kubeapps
28026   1.00  2020-12-27T12:00:32.000000000  github.com/vmware/network-config-manager
26004   1.00  2020-12-10T15:00:59.000000000  github.com/SteeltoeOSS/Documentation
30833   1.00  2021-01-22T10:18:33.000000000  github.com/reactor/reactor-core
25834   1.00  2020-12-07T08:47:40.000000000  github.com/Spring-projects/spring-boot-data-geode
28133   1.00  2021-01-22T09:47:32.000000000  github.com/concourse/ci
25726   0.99  2020-12-29T08:45:41.000000000  github.com/Spring-projects/spring-security
27950   0.98  2020-12-27T14:45:0

In [76]:
print(start_date, end_date)
print('repo_id ratio last_pr_collected              repo_url')
i = 0
for index, repo in top.iterrows():
    repo_id = repo['repo_id']


        #ratio_abs = round(missing_data.loc[missing_data['pull_request_repo_id'] == repo_id]['ratio_abs'].values[0], 2)
    try:
        ratio_abs = float(format(missing_data.loc[missing_data['pull_request_repo_id'] == repo_id]['ratio_abs'].values[0], '.2f'))
    except:
        ratio_abs = 99.0
        
    try:
        last_pr = missing_data.loc[missing_data['pull_request_repo_id'] == repo_id]['last_pr_collected'].values[0]
    except:
        last_pr = 'UNKNOWN'
    
    if ratio_abs < 1:
        repo_info = str(repo_id) + '   ' + str(ratio_abs) + '  ' + str(last_pr)[0:10] + '  ' + repo['repo_path'] + repo['repo_name'] 
        print(repo_info)
        i+=1
    
print('Number of repos:', i)

'2020-07-04' '2021-01-25'
repo_id ratio last_pr_collected              repo_url
27939   0.97  2021-01-15  github.com/vmware/nsx-powerops
27923   0.99  2020-12-07  github.com/vmware/weathervane
28149   0.99  2021-01-15  github.com/goharbor/website
28109   0.99  2020-12-29  github.com/concourse/docs
30605   0.99  2020-12-27  github.com/spring-cloud/spring-cloud-commons
30623   0.99  2021-01-22  github.com/spring-cloud/spring-cloud-openfeign
30489   0.97  2021-01-22  github.com/k14s/ytt
27927   0.98  2020-12-27  github.com/vmware/container-service-extension
25638   0.98  2020-12-29  github.com/spring-io/start.spring.io
28002   0.98  2020-12-27  github.com/vmware/flowgate
25726   0.99  2020-12-29  github.com/Spring-projects/spring-security
27950   0.98  2020-12-27  github.com/vmware/differential-datalog
28150   0.99  2020-12-29  github.com/goharbor/harbor
Number of repos: 13


SyntaxError: invalid syntax (<ipython-input-47-e3c0d519daae>, line 1)

In [49]:
!cat config.json

{
    "connection_string": "sqlite:///:memory:",
    "database": "panthers_project",
    "host": "vista.osshealth.io",
    "password": "UKPivotal",
    "port": 5433,
    "schema": "augur_data",
    "user": "dawn",
    "user_type": "read_only"
}


In [ ]:
check_prs = pd.DataFrame()
check_prs_query = f"""
select 'pull requets collected' as data_point, count(*) as count from pull_requests
union 
SELECT
data_point, 
SUM ( COUNT ) AS COUNT 
FROM
(
SELECT 'platform metadata' as data_point, 
repo_id,
SUM ( last_collected ) AS COUNT 
FROM
( SELECT repo_id, MAX ( data_collection_date ), COUNT ( * ) AS last_collected FROM augur_data.repo_info GROUP BY repo_id ORDER BY repo_id ) A 
GROUP BY
repo_id
) b group by data_point;
"""
check_prs = pd.read_sql_query(check_prs_query, con=engine)
print(check_prs)